
## Modifying a model

Of course, a premade model may not behave in the exact way you want for your application but it may not be different enough to warrant making your own from scratch. In these instances any properties of a model can be modified or changed. To do so we simply call a method which can change or add to a property on either the base model (`label=None`), a specific model (e.g. `label="nebular"`), or (for certain properties) all applicable models (`set_all=True`).

You can add a mask.

In [ ]:
from synthesizer.dust.attenuation import PowerLaw
from synthesizer.dust.emission import Blackbody
from synthesizer.emission_models import TotalEmission
from synthesizer.grid import Grid
from unyt import dimensionless, kelvin

# Get the grid which we'll need for extraction
grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

In [ ]:
# Make a model
model = TotalEmission(
    grid=grid,
    dust_curve=PowerLaw(slope=-1),
    tau_v=0.33,
    fesc=0.2,
    fesc_ly_alpha=0.7,
    dust_emission_model=Blackbody(temperature=100 * kelvin),
)

model.add_mask("log10ages", 7 * dimensionless, "<", label="nebular_continuum")
model.add_mask("log10ages", 7 * dimensionless, "<", label="linecont")

Here we've just added a filter for only young stars for the nebular component. Notice however, that we have had to supply units alongside our threshold, this is required to maintain consistency across objects but also enables arbitrary units to be used for a threshold.

We can see the effect of this in the emission model tree.

In [ ]:
model.plot_emission_tree()



You can change the grid (here we'll just pass the same grid again for demonstration purposes).

In [ ]:
model.set_grid(grid, set_all=True)

Notice that we've changed the grid for the whole model by passing `set_all=True`.

We can also change dust properties but for these we don't have to do each individually and can instead call a single method.

In [ ]:
model.set_dust_props(
    dust_curve=PowerLaw(slope=-0.7),
    apply_dust_to=model["reprocessed"],
    tau_v=0.7,
    label="attenuated",
)

Notice we've used `model["reprocessed"]` to access the reprocessed model that's already part of the `EmissionModel` in the above, an `EmissionModel` supports dicitonary like key indexing to extract models by label.

As well as just changing the properties of existing models we can also replace a model with one or more models. As an example, below we will swap out the attenuation model with two separate attenuation models which apply different dust curves to the old and young population (for further information on defining your own models see below).

In [ ]:
from synthesizer.emission_models import EmissionModel

# Define the models well replace attenuate with
young_attenuated = EmissionModel(
    "young_attenuated",
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=model["reprocessed"],
    tau_v=0.7,
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<=",
)
old_attenuated = EmissionModel(
    "old_attenuated",
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=model["reprocessed"],
    tau_v=0.7 * 0.67,
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op=">",
)

model.replace_model("attenuated", young_attenuated, old_attenuated)
model.plot_emission_tree()

Although not shown here explicitly, passing a single model for the replacement will just swap out the model. 